In [68]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import math
import statistics
from scipy.stats import norm
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn
import matplotlib.mlab as mlab
import scipy

In [69]:
#large VaR Engine Utilizng the Historical Method
list_assets = ["HSBC", "BCS", "LYG", "RBSPF"]
dict_assets = {}

def hour_wiper(df):
    
    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df = df.set_index('Date')
    
    return df

def data_grabber(tickers, assets):
    #Function designed to take any ticker and grab annual data
    for x in tickers:
        try: 
            
            ticker = yf.Ticker(x)
            ticker_data = ticker.history(period = "1y", interval = "1d")
            ticker_data = hour_wiper(ticker_data)
            
            if not ticker_data.empty:
                assets[x] = ticker_data
            else:
                print(f"The associated data for {x} is empty")
        except Exception as e:
            print(f"There was an error in fetching {x} data")
            
    return assets

def returns(dict_assets):
    #Calculates daily returns of each asset in the portfolio and puts them into a seperate dataframe
    for x in dict_assets:
        df_common = dict_assets[x]    
        df_common["Returns"] = df_common["Close"].pct_change()
        df_common = df_common.dropna()        
        dict_assets[x] = df_common
    
    dict_returns = {}    
    for x in dict_assets:
        df_common = dict_assets[x]
        series_returns = df_common["Returns"]
        dict_returns[x] = series_returns
        
    df_returns = pd.concat(dict_returns.values(), axis = 1, keys=dict_returns)
    mean_returns = df_returns.mean()
    
    return dict_assets, df_returns, mean_returns

def weights(dict_assets):
    #Random weight generator 
    array_weights = np.random.random(len(dict_assets.keys()))
    array_weights /= np.sum(array_weights)

    counter = 0

    for key in dict_assets:
        df_common = dict_assets[key]
        df_common["Weights"] = array_weights[counter]
        df_common = df_common.dropna()
        dict_assets[key] = df_common
        counter = counter + 1
        
    return dict_assets, array_weights

def portfolio(dict_assets):
    #Assembles the portfolio into a singular dataframe
    df_portfolio = pd.concat(dict_assets.values(), axis = 1, keys=dict_assets.keys())
    df_portfolio = df_portfolio.reset_index()

    return df_portfolio
    
def portfolio_return(df_portfolio, df_returns, array_weights):
    #Calculates the daily returns of the portfolio given the weights
        
    series_portfolio_return = df_returns.dot(array_weights)
    df_portfolio["Portfolio Returns"] = series_portfolio_return
    
    return df_portfolio, df_returns, series_portfolio_return
    
def covariance_matrix(dict_assets, df_returns):
    #Finds the covariance between the returns of each asset in the portfolio        
    df_cov = df_returns.cov()
    
    return df_cov
  
def portfolio_variance_calculator(df_cov, array_weights):
    #Find portfolio variance given covariance matrix and weights of the portfolio
    array_portfolio_variance = np.dot(array_weights.T, np.dot(df_cov, array_weights))
    float_portfolio_variance = float(array_portfolio_variance)
    return float_portfolio_variance  
    
def portfolio_std(float_portfolio_variance):
    #Multiply by the square root of 252 to annualize the volatility
    float_portfolio_std = (math.sqrt(float_portfolio_variance))*np.sqrt(252)
    
    return float_portfolio_std
    
def portfolio_mean_return(mean_returns, array_weights):
    #Multiply by the number of trading days to annualize average portfolio return
    P_returns = np.sum(mean_returns*array_weights)*252

    
    return P_returns

def add_portfolio_returns_to_df_returns(df_returns, series_portfolio_returns):
    df_returns["Portfolio Returns"] = series_portfolio_returns
    return df_returns

def historicalVaR(returns):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the percentile of the distribution at the given alpha confidence level
    """
    list_alpha = [0.9, 0.95, 0.975, 0.99]
    dict_VaR = {}
       
    if isinstance(returns, pd.Series):
        for x in list_alpha:
            dict_VaR[x] = np.percentile(returns, (1-x)*100)
    elif isinstance(returns, pd.DataFrame):
        for x in list_alpha:
            dict_VaR[x] = returns.apply(lambda col: np.percentile(col, (1-x)*100))
            
    for x in dict_VaR:
        dict_VaR[x] = (dict_VaR[x])["Portfolio Returns"]
    
    display(dict_VaR)
    
    return dict_VaR

def historicalCVaR(returns, dict_VaR):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the CVaR for dataframe / series
    """
    list_alpha = [0.9, 0.95, 0.975, 0.99]
    dict_CVaR = {}

    if isinstance(returns, pd.Series):
        for x in list_alpha:
            belowVaR = returns <= dict_VaR[x]
            CVaR = returns[belowVaR].mean()
            dict_CVaR[x] = CVaR
        
        for x in dict_CVaR:
            dict_CVaR[x] = (dict_CVaR[x])["Portfolio Returns"]
        display(dict_CVaR)
        return dict_CVaR

    # A passed user-defined-function will be passed a Series for evaluation.
    elif isinstance(returns, pd.DataFrame):
        series_portfolio_return = returns["Portfolio Returns"]
        for x in list_alpha:
            belowVaR = series_portfolio_return <= dict_VaR[x]
            CVaR = returns[belowVaR].mean()
            dict_CVaR[x] = CVaR
        for x in dict_CVaR:
            dict_CVaR[x] = (dict_CVaR[x])["Portfolio Returns"]
        display(dict_CVaR)
        return dict_CVaR

    else:
        raise TypeError("Expected returns to be dataframe or series") 



In [70]:
data_grabber(list_assets, dict_assets)
dict_assets, df_returns, mean_returns = returns(dict_assets)
df_portfolio = portfolio(dict_assets)
dict_assets, array_weights = weights(dict_assets)
df_portfolio, df_returns, series_portfolio_returns = portfolio_return(df_portfolio, df_returns, array_weights)
df_cov = covariance_matrix(dict_assets, df_returns)
portfolio_variance = portfolio_variance_calculator(df_cov, array_weights)
std = portfolio_std(portfolio_variance)
Pret = portfolio_mean_return(mean_returns, array_weights)
df_returns = add_portfolio_returns_to_df_returns(df_returns, series_portfolio_returns)
dict_VaR = historicalVaR(df_returns)
dict_CVaR = historicalCVaR(df_returns, dict_VaR)


{0.9: -0.019970533783538014,
 0.95: -0.028701849637319334,
 0.975: -0.037870411359193036,
 0.99: -0.04325239031088311}

{0.9: -0.0312573712620981,
 0.95: -0.03879830407006094,
 0.975: -0.043303352397203274,
 0.99: -0.04864409243960898}